In [4]:
from __future__  import print_function, division
import pylab as pl
import pandas as pd
import numpy as np
import seaborn
import os

__author__= 'JKtours'

%pylab inline

if os.getenv ('PUI2017') is None:
    print ("Must set env variable PUI2017")
if os.getenv ('PUIDATA') is None:
    print ("Must set env variable PUI2017")

import os
import json




Populating the interactive namespace from numpy and matplotlib
Must set env variable PUI2017


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [5]:
def getCitiBikeCSV(datestring):
    print ("Downloading", datestring)
    ### First I will check that it is not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + datestring + "-citibike-tripdata.zip " + os.getenv("PUIDATA"))
            ### unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip")
            ## NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
            os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")

In [6]:
datestring = '201612'
getCitiBikeCSV(datestring)

file in place, you can continue


In [7]:
citibikeDf = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring + '-citibike-tripdata.csv')
# df is the dataframe where the content of the csv file is stored
citibikeDf['date'] = pd.to_datetime(citibikeDf['Start Time'])
# note that with dataframes I can refer to variables as dictionary keys, 
# i.e. citibikeDf['Start Time'] or as attributes: citibikeDf.starttime. 
citibikeDf.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,date
0,528,2016-12-01 00:00:04,2016-12-01 00:08:52,499,Broadway & W 60 St,40.769155,-73.981918,228,E 48 St & 3 Ave,40.754601,-73.971879,26931,Subscriber,1964.0,1,2016-12-01 00:00:04
1,218,2016-12-01 00:00:28,2016-12-01 00:04:06,3418,Plaza St West & Flatbush Ave,40.675021,-73.971115,3358,Garfield Pl & 8 Ave,40.671198,-73.974841,27122,Subscriber,1955.0,1,2016-12-01 00:00:28
2,399,2016-12-01 00:00:39,2016-12-01 00:07:19,297,E 15 St & 3 Ave,40.734232,-73.986923,345,W 13 St & 6 Ave,40.736494,-73.997044,19352,Subscriber,1985.0,1,2016-12-01 00:00:39
3,254,2016-12-01 00:00:44,2016-12-01 00:04:59,405,Washington St & Gansevoort St,40.739323,-74.008119,358,Christopher St & Greenwich St,40.732916,-74.007114,20015,Subscriber,1982.0,1,2016-12-01 00:00:44
4,1805,2016-12-01 00:00:54,2016-12-01 00:31:00,279,Peck Slip & Front St,40.707873,-74.001670,279,Peck Slip & Front St,40.707873,-74.001670,23148,Subscriber,1989.0,1,2016-12-01 00:00:54


In [16]:
#fig = pl.figure(figsize(15,15))

subscriberAvg = citibikeDf['Trip Duration'][citibikeDf['User Type']=='Subscriber'][citibikeDf['Trip Duration'] >= 3600*0.75].sum() /citibikeDf['Trip Duration'][citibikeDf['User Type']=='Subscriber'][citibikeDf['Trip Duration'] >= 0].sum()
customerAvg = citibikeDf['Trip Duration'][citibikeDf['User Type']=='Customer'][citibikeDf['Trip Duration'] >= 3600*0.75].sum() /citibikeDf['Trip Duration'][citibikeDf['User Type']=='Customer'][citibikeDf['Trip Duration'] >= 0].sum()

Comparison = [('Subscriber', subscriberAvg), ('Customers', customerAvg )]
Comparison 

#[citibikeDf['Trip Duration'] >= 3600*0.75].plot

[('Subscriber', 0.10330546386593407), ('Customers', 0.49546588975107975)]

In [17]:


#import numpy as np
#import matplotlib.pyplot as plt

#top = [('a',1.875),('c',1.125),('d',0.5)]

labels, values = zip(*Comparison)
#indexes = np.arange(len(labels))
width = 1

pl.bar(Comparison)
#plt.bar(indexes, values, width)
#plt.xticks(indexes + width * 0.5, labels)
#plt.savefig('netscore.png')

TypeError: bar() takes at least 2 arguments (1 given)

In [10]:
import numpy as np
import matplotlib.pyplot as plt

top = [('a',1.875),('c',1.125),('d',0.5)]

labels, values = zip(*top)
indexes = np.arange(len(labels))
width = 1

plt.bar(indexes, values, width)
plt.xticks(indexes + width * 0.5, labels)
plt.savefig('netscore.png')

70492772

57024705

34926764

70492772

0.49546588975107975

57024705

552000861

Terminology:
Members / Subscribers - are annual members of Citibike program, and pay and annual subscription (along with additional commute charges if applicable)
Customers - pay-per-trip

IDEA: Members of the Citibike program have access to unlimited 45 min commutes for the year, however trips over 45 mins have additional charges. Members (Subscribers) are less likely to travel for over 45 mins than Customers.

NULL HYPOTHESIS: The ratio of members travel over 45 mins as a proportion of members total travel is the same or greater than the ratio of customers travel over 45 mins as a proportion of customers total travel

In [ ]:
citibikeDf['Trip Duration'][citibikeDf['User Type']=='Customer'].mean()
#citibikeDf['Trip Duration'][citibikeDf['User Type']=='Customer'][citibikeDf['Trip Duration'] >= 3600*0.75].sum() #// 
citibikeDf['Trip Duration'][citibikeDf['User Type']=='Customer'][citibikeDf['Trip Duration'] >= 0].sum()
citibikeDf['Trip Duration'][citibikeDf['User Type']=='Subscriber'][citibikeDf['Trip Duration'] >= 3600*0.75].sum()
citibikeDf['Trip Duration'][citibikeDf['User Type']=='Customer'][citibikeDf['Trip Duration'] >= 3600*0.75].sum() 
citibikeDf['Trip Duration'][citibikeDf['User Type']=='Customer'][citibikeDf['Trip Duration'] >= 0].sum()
34926764 / 70492772  # float(70492772)
citibikeDf['Trip Duration'][citibikeDf['User Type']=='Subscriber'][citibikeDf['Trip Duration'] >= 3600*0.75].sum()
citibikeDf['Trip Duration'][citibikeDf['User Type']=='Subscriber'][citibikeDf['Trip Duration'] >= 0].sum()